## Drop *all* entries after 2008
Even if they have been approved.

In [1]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt

import util as u
import pipeline as p
import assess_clf_models as acm
%load_ext autoreload
%autoreload 2

import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=FutureWarning)
warnings.filterwarnings(action='ignore', category=DeprecationWarning)
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

from sklearn.model_selection import train_test_split

### Load Data

In [2]:
if 1 == 0:
    # df0 = u.open_pkl('Data/compound_properties_02_no_dups.pkl')
    df0 = pd.read_csv('Data/compound_properties_02.csv')
    df1 = pd.read_csv('Data/mol_docs_year_syn_res.csv') # num of docs, year-guess, if mol has syn, if mol has res co
    df2 = pd.read_csv('Data/molecule_target_agg_query.csv') # num of targets, target classes, if human_target
    df4 = u.open_pkl('Data/molecule_assays.pkl') # num of activities, num of assays, assay types
    df5 = u.open_pkl('Data/molecule_structure_alerts.pkl') # structural alerts for molecule

    df = pd.merge(df0, df1, how='left', on='molecule_key')
    df = pd.merge(df, df2, how='left', on='molecule_key')
    df = pd.merge(df, df4, how='left', on='molecule_key')
    df = pd.merge(df, df5, how='left', on='molecule_key')
    df = u.clean_model_df(df)
    u.pkl_this('Data/model_df_02.pkl', df)
else:
    df = u.open_pkl('Data/model_df_02.pkl')

In [3]:
df.head()

,molecule_key,molecule_name,mw_freebase,alogp,acd_logp,acd_logd,hba,hbd,psa,rtb,...,num_assays,assay_types,num_alerts_set1,alert_yn_set1,num_alerts_set8,alert_yn_set8,alerts_set1,alerts_set8,patent,approved
0,460508,none given,691.65,1.92,1.69,1.69,13.0,8.0,222.86,6.0,...,6,"B,F",0,0,0,0,[None],[None],0,0
1,506107,none given,934.63,1.00,1.70,-4.57,26.0,15.0,452.00,2.0,...,2,F,1,1,0,0,"[I7 2,4,5 trihydroxyphenyl]",[None],0,0
2,623893,none given,913.88,9.13,-0.23,-5.75,18.0,7.0,372.91,11.0,...,2,B,0,0,4,1,[None],"[acyclic N-,=N and not N bound to carbonyl or ...",0,0
3,460509,none given,691.65,1.92,1.81,1.81,13.0,8.0,222.86,6.0,...,6,"B,F",0,0,0,0,[None],[None],0,0
4,47659,none given,810.68,6.12,2.12,-3.13,16.0,8.0,282.68,6.0,...,59,"B,F",0,0,0,0,[None],[None],0,0


In [3]:
len(df)

1699286

# Clean Data
- Drop duplicate entries
- Drop missing years & year > 2008, but keep phase > 0
    - No reliable way to impute
    - Keep those who we know the outcome for (i.e. approved = 1). This is 300 entries. Consider it a form of over-sampling
- Drop n/a's for compound_properties columns

In [4]:
cols_to_compare = ['mw_freebase', 'alogp', 'acd_logp',
       'acd_logd', 'hba', 'hbd', 'psa', 'rtb', 'ro3_pass',
       'num_ro5_violations', 'acd_most_apka', 'acd_most_bpka',
       'molecular_species', 'aromatic_rings', 'heavy_atoms', 'qed_weighted',
       'hba_lipinski', 'hbd_lipinski', 'num_lipinski_ro5_violations']

df_no_dups = df.sort_values('num_assays', ascending=False).drop_duplicates(subset=cols_to_compare, keep='first')

len(df_no_dups)

1522351

In [5]:
df_drop = df_no_dups[(df_no_dups['year_impute'].notnull())] #| (df_no_dups['approved'] == 1)]
df_drop = df_drop[(df_drop['year_impute'] < 2009) ]#| (df_drop['approved'] == 1)]

cols_to_drop_nas = ['alogp', 'acd_logp', 'acd_logd', 'hba', 'hbd', 'psa', 'rtb', 'ro3_pass',
       'num_ro5_violations', 'molecular_species', 'aromatic_rings', 'heavy_atoms', 'qed_weighted',
       'hba_lipinski', 'hbd_lipinski']

df_drop = df_drop.dropna(axis=0, subset=cols_to_drop_nas)

print(len(df_drop))

432160


In [6]:
df_drop['approved'].value_counts(1)

0    0.99577
1    0.00423
Name: approved, dtype: float64

This is a better perecentage, actually. Use this data instead?

In [7]:
df_drop['phase'].value_counts()

0    429976
4      1544
3       284
2       269
1        87
Name: phase, dtype: int64

In [9]:
1544 + 284

1828

In [10]:
1544 + 284 + 269 + 87

2184

In [12]:
1.8e7 / 2000

9000.0

In [13]:
43000 / 2000

21.5

In [8]:
df_drop['phase'].value_counts(1)

0    0.994946
4    0.003573
3    0.000657
2    0.000622
1    0.000201
Name: phase, dtype: float64

# Create test-train split

In [14]:
y1 = df_drop['approved']
X1 = df_drop.drop(columns='approved')
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.33, random_state=19)
X_train1.reset_index(inplace=True, drop=True)

In [15]:
u.pkl_this('Data/X_train_10yrs_02.pkl', X_train1)
u.pkl_this('Data/y_train_10yrs_02.pkl', y_train1)
u.pkl_this('Data/X_test_10yrs_02.pkl', X_test1)
u.pkl_this('Data/y_test_10yrs_02.pkl', y_test1)